In [1]:
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path = "/content/drive/My Drive"

Mounted at /content/drive


In [14]:
os.chdir("/content/drive/My Drive/UAS/code")

In [4]:
# !pip list

In [5]:
os.getcwd()

'/content/drive/My Drive/UAS/code'

In [6]:
# pip install tensorflow.contrib.slim

**Uninstall Tensorflow v2.7.0 dari google colaboratory untuk direplace dengan Tensorflow v1**

In [7]:
pip uninstall tensorflow

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0


**Install Tensorflow v1 untuk menjalankan code (Script di bawah hanya support menggunakan TFv1)**

In [8]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7 MB 1.2 MB/s 
     |████████████████████████████████| 367 kB 64.1 MB/s 
     |████████████████████████████████| 3.2 MB 54.0 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, but you have tensorflow 1.13.2 which is incompatible.


TensorShape([Dimension(None), Dimension(400)])

# NET.py

**Pada code ini, membuat sebuah fungsi sebagai base model yang akan digunakan**

In [10]:
# author is He Zhao
# The time to create is 8:47 PM, 28/11/16

import tensorflow as tf
import tensorflow.contrib.slim as slim
from Opts import lrelu, resUnit
from dataBlocks import DataBlocks
# import cPickle
import pickle
import scipy.io as sio
import numpy as np


initializer = tf.truncated_normal_initializer(stddev=0.02)
bias_initializer = tf.constant_initializer(0.0)


def build_data(batchsize):
    readpath = open('img.pkl', 'rb')
    datapaths = pickle.load(readpath)
    db = DataBlocks(data_paths=datapaths, train_valid_ratio=[39, 0], batchsize=batchsize, allow_preload=False)
    mask_s = np.load('syn_data/mask.npy')
    return db, mask_s
   


def matTonpy_35():

    img = sio.loadmat('test_1To4.mat')['imgAllTest']
    gt = sio.loadmat('test_1To4.mat')['gtAllTest']
    mask = sio.loadmat('test_1To4.mat')['maskAllTest']

    return img, gt, mask

def discriminator(image, reuse=False):
    n=32
    bn = slim.batch_norm
    with tf.name_scope("disciminator"):
        # original
        dis1 = slim.convolution2d(image, n, [4, 4], 2, activation_fn=lrelu,
                                  reuse=reuse, scope='d_conv1', weights_initializer=initializer)

        dis2 = slim.convolution2d(dis1, 2*n, [4, 4], 2, normalizer_fn=bn, activation_fn=lrelu,
                                  reuse=reuse, scope='d_conv2', weights_initializer=initializer)

        dis3 = slim.convolution2d(dis2, 4*n, [4, 4], 2, normalizer_fn=bn, activation_fn=lrelu,
                                  reuse=reuse, scope='d_conv3', weights_initializer=initializer)

        dis4 = slim.convolution2d(dis3, 8*n, [4, 4], 2, normalizer_fn=bn, activation_fn=lrelu,
                                  reuse=reuse, scope='d_conv4', weights_initializer=initializer)

        dis5 = slim.convolution2d(dis4, 16*n, [4, 4], 2, normalizer_fn=bn, activation_fn=lrelu,
                                  reuse=reuse, scope='d_conv5', weights_initializer=initializer)

        
        d_out_logits = slim.fully_connected(slim.flatten(dis5), 1, activation_fn=None, reuse=reuse, scope='d_out',
                                            weights_initializer=initializer)

        d_out = tf.nn.sigmoid(d_out_logits)
    return d_out, d_out_logits
    
def generator(image, z):
    n = 64
    with tf.name_scope("generator"):
        # original
        e1 = slim.conv2d(image, n, [4, 4], 2, activation_fn=lrelu,
                         weights_initializer=initializer)
        # 256
        e2 = slim.conv2d(lrelu(e1), 2 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                         weights_initializer=initializer)
        # 128
        e3 = slim.conv2d(lrelu(e2), 4 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                         weights_initializer=initializer)
        # 64
        e4 = slim.conv2d(lrelu(e3), 8 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                         weights_initializer=initializer)
        # 32
        e5 = slim.conv2d(lrelu(e4), 8*n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                         weights_initializer=initializer)
        # 16
        e6 = slim.conv2d(lrelu(e5), 8*n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                         weights_initializer=initializer)
        # e1 = slim.conv2d(image, n, [4, 4], 2, activation_fn=lrelu, scope='g_e1_conv',
        #                  weights_initializer=initializer, reuse=True)
        # # 256
        # e2 = slim.conv2d(lrelu(e1), 2 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None, scope='g_e2_conv',
        #                  weights_initializer=initializer,reuse=True)
        # # 128
        # e3 = slim.conv2d(lrelu(e2), 4 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None, scope='g_e3_conv',
        #                  weights_initializer=initializer,reuse=True)
        # # 64
        # e4 = slim.conv2d(lrelu(e3), 8 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None, scope='g_e4_conv',
        #                  weights_initializer=initializer,reuse=True)
        # # 32
        # e5 = slim.conv2d(lrelu(e4), 8*n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None, scope='g_e5_conv',
        #                  weights_initializer=initializer,reuse=True)
        # # 16
        # e6 = slim.conv2d(lrelu(e5), 8*n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None, scope='g_e6_conv',
        #                  weights_initializer=initializer,reuse=True)



        zP = slim.fully_connected(z, 4 * 4 * n, normalizer_fn=None, activation_fn=lrelu, scope='g_project',
                                  weights_initializer=initializer,reuse=True)
        zCon = tf.reshape(zP, [-1, 4, 4, n])

        gen1 = slim.conv2d_transpose(lrelu(zCon), 2 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                     scope='g_dconv1', weights_initializer=initializer,reuse=True)
        # 8
        gen1 = tf.concat(3, [gen1, e6])
        
        gen2 = slim.conv2d_transpose(lrelu(gen1), 4 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                     scope='g_dconv2', weights_initializer=initializer,reuse=True)
        # 16
        gen2 = tf.concat(3, [gen2, e5])

        gen3 = slim.conv2d_transpose(lrelu(gen2), 8 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                     scope='g_dconv3', weights_initializer=initializer,reuse=True)
        gen3 = tf.concat(3, [gen3, e4])

        # 32
        gen6 = slim.conv2d_transpose(tf.nn.relu(gen3), 4 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                     scope='g_dconv6', weights_initializer=initializer,reuse=True)
        gen6 = tf.concat(3, [gen6, e3])

        # 64
        gen7 = slim.conv2d_transpose(tf.nn.relu(gen6), 2 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                     scope='g_dconv7', weights_initializer=initializer,reuse=True)
        gen7 = tf.concat(3, [gen7, e2])

        # 128
        gen8 = slim.conv2d_transpose(tf.nn.relu(gen7), n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                     scope='g_dconv8', weights_initializer=initializer,reuse=True)
        # gen8 = tf.nn.dropout(gen8, 0.5)
        gen8 = tf.concat([gen8, e1],3)
        gen8 = tf.nn.relu(gen8)

        # 256
        gen_out = slim.conv2d_transpose(gen8, 3, [4, 4], 2, activation_fn=tf.nn.tanh, scope='g_out',
                                        weights_initializer=initializer)

    return gen_out    
    

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
# dis1 = slim.convolution2d(img, n, [4, 4], 2, activation_fn=lrelu, weights_initializer=initializer)

In [ ]:
# author is He Zhao
# The time to create is 8:47 PM, 28/11/16

import tensorflow as tf
import tensorflow.contrib.slim as slim
from Opts import lrelu, resUnit
from dataBlocks import DataBlocks
# import cPickle
import pickle
import scipy.io as sio
import numpy as np


initializer = tf.truncated_normal_initializer(stddev=0.02)
bias_initializer = tf.constant_initializer(0.0)


def build_data(batchsize):
    readpath = open('img.pkl', 'rb')
    datapaths = pickle.load(readpath)
    db = DataBlocks(data_paths=datapaths, train_valid_ratio=[39, 0], batchsize=batchsize, allow_preload=False)
    mask_s = np.load('syn_data/mask.npy')
    return db, mask_s
   


def matTonpy_35():

    img = sio.loadmat('test_1To4.mat')['imgAllTest']
    gt = sio.loadmat('test_1To4.mat')['gtAllTest']
    mask = sio.loadmat('test_1To4.mat')['maskAllTest']

    return img, gt, mask

def discriminator(image, reuse=False):
    n=32
    bn = slim.batch_norm
    with tf.name_scope("disciminator"):
        # original
        dis1 = slim.convolution2d(image, n, [4, 4], 2, activation_fn=lrelu,
                                  reuse=reuse, weights_initializer=initializer)

        dis2 = slim.convolution2d(dis1, 2*n, [4, 4], 2, normalizer_fn=bn, activation_fn=lrelu, weights_initializer=initializer)

        dis3 = slim.convolution2d(dis2, 4*n, [4, 4], 2, normalizer_fn=bn, activation_fn=lrelu, weights_initializer=initializer)

        dis4 = slim.convolution2d(dis3, 8*n, [4, 4], 2, normalizer_fn=bn, activation_fn=lrelu, weights_initializer=initializer)

        dis5 = slim.convolution2d(dis4, 16*n, [4, 4], 2, normalizer_fn=bn, activation_fn=lrelu, weights_initializer=initializer)

        
        d_out_logits = slim.fully_connected(slim.flatten(dis5), 1, activation_fn=None, reuse=reuse,weights_initializer=initializer)

        d_out = tf.nn.sigmoid(d_out_logits)
    return d_out, d_out_logits
    
def generator(image, z):
    n = 64
    with tf.name_scope("generator"):
        # original
        e1 = slim.conv2d(image, n, [4, 4], 2, activation_fn=lrelu,
                  weights_initializer=initializer)
        # 256
        e2 = slim.conv2d(lrelu(e1), 2 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                          weights_initializer=initializer)
        # 128
        e3 = slim.conv2d(lrelu(e2), 4 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                          weights_initializer=initializer)
        # 64
        e4 = slim.conv2d(lrelu(e3), 8 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                          weights_initializer=initializer)
        # 32
        e5 = slim.conv2d(lrelu(e4), 8*n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                          weights_initializer=initializer)
        # 16
        e6 = slim.conv2d(lrelu(e5), 8*n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                          weights_initializer=initializer)



        zP = slim.fully_connected(z, 4 * 4 * n, normalizer_fn=None, activation_fn=lrelu, scope='g_project',
                          weights_initializer=initializer,reuse=True)
        zCon = tf.reshape(zP, [-1, 4, 4, n])

        gen1 = slim.conv2d_transpose(lrelu(zCon), 2 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                      scope='g_dconv1', weights_initializer=initializer,reuse=True)
        # # 8
        gen1 = tf.concat([gen1, e6],3)

        gen2 = slim.conv2d_transpose(lrelu(gen1), 4 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                      scope='g_dconv2', weights_initializer=initializer,reuse=tf.AUTO_REUSE)
        # 16
        gen2 = tf.concat([gen2, e5],3)

        gen3 = slim.conv2d_transpose(lrelu(gen2), 8 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                      scope='g_dconv3', weights_initializer=initializer,reuse=tf.AUTO_REUSE)
        gen3 = tf.concat([gen3, e4],3)

        # 32
        gen6 = slim.conv2d_transpose(tf.nn.relu(gen3), 4 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                      scope='g_dconv6', weights_initializer=initializer,reuse=tf.AUTO_REUSE)
        gen6 = tf.concat([gen6, e3],3)

        # 64
        gen7 = slim.conv2d_transpose(tf.nn.relu(gen6), 2 * n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                      scope='g_dconv7', weights_initializer=initializer,reuse=tf.AUTO_REUSE)
        gen7 = tf.concat([gen7, e2],3)

        # 128
        gen8 = slim.conv2d_transpose(tf.nn.relu(gen7), n, [4, 4], 2, normalizer_fn=slim.batch_norm, activation_fn=None,
                                      scope='g_dconv8', weights_initializer=initializer,reuse=tf.AUTO_REUSE)
        # gen8 = tf.nn.dropout(gen8, 0.5)
        gen8 = tf.concat([gen8, e1],3)
        gen8 = tf.nn.relu(gen8)

        # 256
        gen_out = slim.conv2d_transpose(gen8, 3, [4, 4], 2, activation_fn=tf.nn.tanh, scope='g_out',
                                        weights_initializer=initializer,reuse=tf.AUTO_REUSE)

    return gen_out    
    

# OPTS.py

In [ ]:
# author is He Zhao
# The time to create is 8:49 PM, 28/11/16

import tensorflow as tf
import numpy as np
import scipy.io as sio
import scipy.misc
import scipy
import tensorflow.contrib.slim as slim


def lrelu(x, leak=0.2, name='lrelu'):
    with tf.variable_scope(name):
        f1 = 0.5 * (1 + leak)
        f2 = 0.5 * (1 - leak)
        return f1 * x + f2 * abs(x)

def lrelu1(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)


def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)


def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image

    return img


def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))


def inverse_transform(images):
    return (images+1.)/2.

def matTonpy():

    # img = sio.loadmat('sample.mat')['imgSample']
    # gt = sio.loadmat('sample.mat')['gtSample']

    img = sio.loadmat('test.mat')['imgAllTest']
    gt = sio.loadmat('test.mat')['gtAllTest']
		
    # with open('img_sample.npy', 'wb') as fout:
    #     np.save(fout, img)
    # with open('gt_sample.npy', 'wb') as fout:
    #     np.save(fout, gt)
    return img, gt


def TestImgForTest(dataPath):

    img = sio.loadmat(dataPath)['imgAllTest']
    gt = sio.loadmat(dataPath)['gtAllTest']

    return img, gt


def TrainImgForTest(dataPath):

    img = sio.loadmat(dataPath)['imgAllTrain']
    gt = sio.loadmat(dataPath)['gtAllTrain']

    return img, gt

def resUnit(input_layer, i, out_size):
    with tf.variable_scope("g_res_unit" + str(i)):
        net = slim.conv2d(inputs=input_layer, normalizer_fn=slim.batch_norm, activation_fn=lrelu,
                          num_outputs=out_size, kernel_size=[4, 4], stride=2, padding='SAME')

        net = slim.conv2d(inputs=net, normalizer_fn=slim.batch_norm, activation_fn=lrelu,
                          num_outputs=out_size, kernel_size=[4, 4], stride=1, padding='SAME')

        res = slim.conv2d(inputs=input_layer, normalizer_fn=slim.batch_norm, activation_fn=lrelu,
                          num_outputs=out_size, kernel_size=[1, 1], stride=2, padding='SAME')

        output = net + res
    return output


def resUnit_up(input_layer, i, out_size):
    with tf.variable_scope("g_res_unit_up" + str(i)):
        net = slim.conv2d_transpose(inputs=input_layer, normalizer_fn=slim.batch_norm, activation_fn=lrelu,
                                    num_outputs=out_size, kernel_size=[4, 4], stride=1, padding='SAME')

        net = slim.conv2d_transpose(inputs=net, normalizer_fn=slim.batch_norm, activation_fn=lrelu,
                                    num_outputs=out_size, kernel_size=[4, 4], stride=2, padding='SAME')

        res = slim.conv2d_transpose(inputs=input_layer, normalizer_fn=slim.batch_norm, activation_fn=lrelu,
                                    num_outputs=out_size, kernel_size=[1, 1], stride=2, padding='SAME')

        output = net + res
    return output

# VGG.py

**Code di bawah ini merupakan sebuah fungsi untuk model deep neural network dengan arsitektur VGG**

In [ ]:
# Copyright (c) 2015-2016 Anish Athalye. Released under GPLv3.

import tensorflow as tf
import numpy as np
import scipy.io
import pdb

MEAN_PIXEL = np.array([ 123.68 ,  116.779,  103.939])

def net(data, input_image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',

        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )

    data = scipy.io.loadmat(data_path)
    mean = data['normalization'][0][0][0]
    mean_pixel = np.mean(mean, axis=(0, 1))
    weights = data['layers'][0]

    net = {}
    current = input_image
    for i, name in enumerate(layers):
        kind = name[:4]
        if kind == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
        elif kind == 'relu':
            current = tf.nn.relu(current)
        elif kind == 'pool':
            current = _pool_layer(current)
        net[name] = current

    assert len(net) == len(layers)
    return net


def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME')
    return tf.nn.bias_add(conv, bias)


def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
            padding='SAME')


def preprocess(image):
    return image - MEAN_PIXEL


def unprocess(image):
    return image + MEAN_PIXEL

# Utils.py

**Script di bawah ini untuk memanggil input gambar ke dalam model dan melakukan transformasi pada input (gambar)**

In [19]:
import scipy.misc, numpy as np, os, sys
import imageio

def save_img(out_path, img):
    img = np.clip(img, 0, 255).astype(np.uint8)
    scipy.misc.imsave(out_path, img)

def scale_img(style_path, style_scale):
    scale = float(style_scale)
    # o0, o1, o2 = scipy.misc.imread(style_path, mode='RGB').shape
    o0, o1, o2 = imageio.imread(style_path, mode='RGB').shape
    scale = float(style_scale)
    new_shape = (int(o0 * scale), int(o1 * scale), o2)
    style_target = get_img(style_path, img_size=new_shape)
    return style_target

def get_img(src, img_size=False):
  #  img = scipy.misc.imread(src, mode='RGB') # misc.imresize(, (256, 256, 3))
   img = imageio.imread(src)
   if not (len(img.shape) == 3 and img.shape[2] == 3):
       img = np.dstack((img,img,img))
   if img_size != False:
       img = scipy.misc.imresize(img, img_size)
   return img

def exists(p, msg):
    assert os.path.exists(p), msg

def list_files(in_path):
    files = []
    for (dirpath, dirnames, filenames) in os.walk(in_path):
        files.extend(filenames)
        break

    return files

In [20]:
get_img(vgg_path)

Array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

# StyleFeature.py

In [ ]:
os.getcwd()

'/content/drive/My Drive/UAS/code'

In [12]:
# vgg_path = "/content/drive/My Drive/UAS/2D_Neuron_dataset/NeuB2/Color_GT/39_color.tif"
# from PIL import Image
# img = Image.open(vgg_path).convert('L').resize((28, 28), Image.ANTIALIAS)
# img = np.array(img)

In [ ]:
# data = image.load_img(vgg_path)

In [16]:

# author is He Zhao
# The time to create is 3:40 PM, 23/3/17
import tensorflow as tf
import vgg
import numpy as np
import scipy.io as sio


STYLE_LAYERS = ('relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1')
CONTENT_LAYER = ('relu4_2',)


# vgg_path = 'imagenet-vgg-verydeep-19.mat'
vgg_path = "/content/drive/My Drive/UAS/2D_Neuron_dataset/NeuB2/Color_GT/39_color.tif"
img = Image.open(vgg_path).convert('L').resize((28, 28), Image.ANTIALIAS)
img = np.array(img)
data = img

def get_style_features(image, mask):

    image = tf.mul(image+1, 127.5)
    image = image*((mask+1)/2)
    
    if image._shape_as_list()[1] != 512:
        image = tf.image.resize_images(image, [512,512])

    img_features = {}

    #with tf.device('/cpu:0'):

    img_pre = vgg.preprocess(image)
    net = vgg.net(data, img_pre)

    for layer in STYLE_LAYERS:
        features = net[layer]
        features = tf.reshape(features, shape=[-1, features._shape_as_list()[1]*features._shape_as_list()[2], features._shape_as_list()[3]])[0]
        features_T = tf.transpose(features)
        gram = tf.matmul(features_T, features) / float(features._shape_as_list()[0]*features._shape_as_list()[1])
        img_features[layer] = gram

    return img_features
    
def get_content_features(image,mask):

    image = tf.mul(image + 1, 127.5)
    image = image*((mask+1)/2)

    img_features = {}

    if image._shape_as_list()[1] != 512:
        image = tf.image.resize_images(image, [512,512])
    
    #with tf.device('/cpu:0'):

    img_pre = vgg.preprocess(image)
    net = vgg.net(data, img_pre)

    for layer in CONTENT_LAYER:
        features = net[layer]
        img_features[layer] = features

    return img_features


def get_style_loss(style_features, img, mask):

    img_features = get_style_features(img, mask)      
    

    style_lossE = 0
    for style_layer in STYLE_LAYERS:
        coff = float(1.0 / len(STYLE_LAYERS))
        img_gram = img_features[style_layer]
        style_gram = style_features[style_layer]
        style_lossE += coff * tf.reduce_mean(tf.abs(img_gram - style_gram))

    style_loss = tf.reduce_mean(style_lossE)

    return style_loss
    

def get_content_loss(img, syn, mask):

    img_features = get_content_features(img, mask)
    syn_features = get_content_features(syn, mask)

    content_lossE = 0
    for content_layer in CONTENT_LAYER:
        coff = float(1.0 / len(CONTENT_LAYER))
        img_content = img_features[content_layer]
        syn_content = syn_features[content_layer]
        content_lossE += coff * tf.reduce_mean(tf.abs(img_content - syn_content))

    content_loss = tf.reduce_mean(content_lossE)

    return content_loss


def get_tv_loss(img, mask):
    img = img*((mask+1)/2)
    # x = tf.reduce_sum(tf.abs(img[:, 1:, :, :] - img[:, :-1, :, :]))
    # y = tf.reduce_sum(tf.abs(img[:, :, 1:, :] - img[:, :, :-1, :]))

    x = tf.reduce_mean(tf.abs(img[:, 1:, :, :] - img[:, :-1, :, :]))
    y = tf.reduce_mean(tf.abs(img[:, :, 1:, :] - img[:, :, :-1, :]))

    return x+y

In [ ]:
gt = tf.placeholder(shape=[None, img_size, img_size, 1], dtype=tf.float32)
img = tf.placeholder(shape=[None, img_size, img_size, channel], dtype=tf.float32)
mask = tf.placeholder(shape=[None, img_size, img_size, 1], dtype=tf.float32)
style = tf.placeholder(shape=[None, style_size, style_size, 3],dtype=tf.float32)
z = tf.placeholder(shape=[None, z_size], dtype=tf.float32)

In [ ]:
print(gt.shape,mask.shape)

(?, 512, 512, 1) (?, 512, 512, 1)


# Train.py

In [21]:
import tensorflow as tf
from tensorflow.python.training import training_util
import Net
import numpy as np
import os
import time
import StyleFeature
import scipy.io as sio
import pdb
#import matplotlib.pyplot as plt

from Opts import save_images, matTonpy
# =============================== path set =============================================== #
load_model = None#'initial_model'
save_model = False

# ============================== parameters set ========================================== #
#adversarial
L1 = 1
#style
L2 = 10
#content
L3 = 1
#tv
L4 = 100
#style number
style_flag = 'drive'
styleNum = 0

# ============================== model set ========================================== #
# model = 'test'

# result_dir = 'Model_and_Result' + '/' + model + ''
# sample_directory = result_dir + '/figs'
# sample_directory2 = result_dir + '/figs_mask'
# # Directory to save sample images from generator in.
# model_directory = result_dir + '/models'  # Directory to save trained model to.

# if tf.gfile.Exists(result_dir):
#     tf.gfile.DeleteRecursively(result_dir)
# if not os.path.exists(sample_directory):
#     os.makedirs(sample_directory)
# if not os.path.exists(sample_directory2):
#     os.makedirs(sample_directory2)
# if not os.path.exists(model_directory):
#     os.makedirs(model_directory)

# os.system('cp {} {}'.format(__file__, result_dir))
# os.system('cp {} {}'.format('Net.py', result_dir))
# os.system('cp {} {}'.format('Opts.py', result_dir))
# os.system('cp {} {}'.format('StyleFeature.py', result_dir))

# with open(model_directory + '/training_log.txt', 'w') as f:
#     f.close()
# ============================== parameters set ========================================== #


learning_rate = 0.0002
beta1 = 0.5

batch_size = 1  # Size of image batch to apply at each iteration.
max_epoch = 100

channel = 3
img_size = 512
img_x = 512
img_y = 512
padding_l = 0
padding_r = 0
padding_t = 0
padding_d = 0

style_size = 512

sample_batch = 4
z_size = 400



# =============================== model and data definition ================================ #
generator = Net.generator
discriminator = Net.discriminator

build_data = Net.build_data

tf.reset_default_graph()

gt = tf.placeholder(shape=[None, img_size, img_size, 1], dtype=tf.float32)
img = tf.placeholder(shape=[None, img_size, img_size, channel], dtype=tf.float32)
mask = tf.placeholder(shape=[None, img_size, img_size, 1], dtype=tf.float32)
style = tf.placeholder(shape=[None, style_size, style_size, 3],dtype=tf.float32)
z = tf.placeholder(shape=[None, z_size], dtype=tf.float32)

# gt_mask = tf.concat(3, [gt, mask])
gt_mask = tf.concat([gt, mask],3)

syn = generator(gt_mask, z)

real_img_gt = tf.concat(3, [img, gt, mask])
fake_syn_gt = tf.concat(3, [syn, gt, mask])

Dx, Dx_logits = discriminator(real_img_gt)
Dg, Dg_logits = discriminator(fake_syn_gt, reuse=True)

db, mask_s = build_data(batch_size)

# ============================= loss function and optimizer =============================== #
# style_features
import vgg
STYLE_LAYERS = ('relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1')
vgg_path = 'imagenet-vgg-verydeep-19.mat'
data = sio.loadmat(vgg_path)

if style_flag == 'drive':
    ss = sio.loadmat('trn20+tst20.mat')['imgAllTrain'][styleNum]
elif style_flag == 'messidor':
    ss = sio.loadmat('style.mat')['imgAll'][styleNum]
elif style_flag == 'stare':
    ss = sio.loadmat('stare_original_mask_binary.mat')['imgAll'][styleNum]
    
ss = (np.reshape(ss, [batch_size, img_size, img_size, 3]) - 0.5) * 2.0
ss = np.lib.pad(ss, ((0, 0), (padding_l, padding_r), (padding_t, padding_d), (0, 0)), 'constant',
                constant_values=(-1, -1))  # Pad the images so the are 32x32

image = (ss + 1) * 127.5

style_features = {}
style_features_x = {}
style_features_y = {}

with tf.Graph().as_default(), tf.device('/cpu:0'), tf.Session() as sess:
    style_image = tf.placeholder(tf.float32, shape=image.shape, name='style_image')
    img_pre = vgg.preprocess(style_image)
    net = vgg.net(data, img_pre)

    for layer in STYLE_LAYERS:
        features = net[layer].eval(feed_dict={style_image:image})
        
        
        features_x = features[:, 1:, :, :] - features[:, :-1, :, :]
        features_y = features[:, :, 1:, :] - features[:, :, :-1, :]

        features_x = np.reshape(features_x, (-1, features_x.shape[1] * features_x.shape[2], features_x.shape[3]))[0]
        features_y = np.reshape(features_y, (-1, features_y.shape[1] * features_y.shape[2], features_y.shape[3]))[0]

        gram_x = np.matmul(features_x.T, features_x) / float(features_x.size)
        gram_y = np.matmul(features_y.T, features_y) / float(features_y.size)
        style_features_x[layer] = gram_x
        style_features_y[layer] = gram_y

        features = np.reshape(features, (-1, features.shape[1] * features.shape[2], features.shape[3]))[0]

        # mean_value = np.mean(features)
        # features = features - mean_value

        gram = np.matmul(features.T, features) / float(features.size)
        style_features[layer] = gram

# ============================================================================================#
# discriminator loss
with tf.name_scope('d_loss'):
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(Dx_logits, tf.ones_like(Dx)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(Dg_logits, tf.zeros_like(Dg)))
    d_loss = d_loss_real + d_loss_fake

# generator loss
with tf.name_scope('g_loss'):

    g_loss_adversarial = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(Dg_logits, tf.ones_like(Dg)))    

    g_loss_style = StyleFeature.get_style_loss(style_features, syn, mask)      
    g_loss_content = StyleFeature.get_content_loss(img, syn, mask)    
   
    g_loss_tv = StyleFeature.get_tv_loss(syn, mask)
    
    g_loss = L1*g_loss_adversarial + L2*g_loss_style + L3*g_loss_content + L4*g_loss_tv 

    
# split the variable for two differentiable function
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'd_' in var.name]
g_vars = [var for var in t_vars if 'g_' in var.name]

# optimizer
global_step = tf.Variable(0, trainable=False)
with tf.name_scope('train'):
    d_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate*0.4).minimize(d_loss, var_list=d_vars, global_step=global_step)
    g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1).minimize(g_loss, var_list=g_vars)

# =============================== summary prepare ============================================= #

# write summary
Dx_sum = tf.histogram_summary("Dx", Dx)
Dg_sum = tf.histogram_summary("Dg", Dg)

Dx_sum_scalar = tf.scalar_summary("Dx_value", tf.reduce_mean(Dx))
Dg_sum_scalar = tf.scalar_summary("Dg_value", tf.reduce_mean(Dg))

syn_sum = tf.image_summary("synthesize", syn)

d_loss_real_sum = tf.scalar_summary("d_loss_real", d_loss_real)
d_loss_fake_sum = tf.scalar_summary("d_loss_fake", d_loss_fake)
d_loss_sum = tf.scalar_summary("d_loss", d_loss)
g_loss_sum = tf.scalar_summary("g_loss", g_loss)

# g_sum = tf.merge_summary([Dg_sum, syn_sum, d_loss_fake_sum, g_loss_sum])
# d_sum = tf.merge_summary([Dx_sum, d_loss_real_sum, d_loss_fake_sum, d_loss_sum])

g_sum = tf.merge_summary([Dg_sum_scalar])
d_sum = tf.merge_summary([Dx_sum_scalar])

# =============================== train phase ============================================= #
init = tf.initialize_all_variables()
sess = tf.Session()
saver = tf.train.Saver(max_to_keep=None)

sess.run(init)
#writer = tf.train.SummaryWriter(model_directory, sess.graph)

# ==================================== save initialization ================================ #
if load_model:
    ckpt = tf.train.get_checkpoint_state('Model_and_Result/' + load_model + '/models')
    saver.restore(sess, ckpt.model_checkpoint_path)
    # saver.save(sess, model_directory + '/model-' + str(0) + '.cptk')
    print ("load saved model and SAVE")
elif save_model:
    saver.save(sess, model_directory + '/model-' + str(0) + '.cptk')
    print ("Saved begining Model ")

# ==================================== start training ===================================== #
stime=time.time()
for epoch in xrange(max_epoch):
    batchNum = 1

    for data_train, _ in db:
        for batch in data_train:

            ms = mask_s[batchNum-1]
            ms = (np.reshape(ms, [batch_size, img_size, img_size, 1]) - 0.5) * 2.0
            ms = np.lib.pad(ms, ((0, 0), (padding_l, padding_r), (padding_t, padding_d), (0, 0)), 'constant',
                            constant_values=(-1, -1))

            z_sample = np.random.normal(0, 0.001, size=[batch_size, z_size]).astype(np.float32)#mean
            zs = z_sample

            xs, ys = batch  # Draw a sample batch from MNIST dataset.
            if xs.shape[0] != batch_size:
                continue

            # xs = np.transpose(xs, (0, 2, 3, 1))
            xs = (np.reshape(xs, [batch_size, img_size, img_size, channel]) - 0.5) * 2.0
            xs = np.lib.pad(xs, ((0, 0), (padding_l, padding_r), (padding_t, padding_d), (0, 0)), 'constant',
                            constant_values=(-1, -1))  # Pad the images so the are 32x32

            # ys = np.transpose(ys, (0, 2, 3, 1))
            ys = (np.reshape(ys, [batch_size, img_size, img_size, 1]) - 0.5) * 2.0
            ys = np.lib.pad(ys, ((0, 0), (padding_l, padding_r), (padding_t, padding_d), (0, 0)), 'constant',
                            constant_values=(-1, -1))  # Pad the images so the are 32x32

            ss = sio.loadmat('test_1To4.mat')['imgAllTest'][3]
            ss = (np.reshape(ss, [batch_size, img_size, img_size, 3]) - 0.5) * 2.0
            ss = np.lib.pad(ss, ((0, 0), (padding_l, padding_r), (padding_t, padding_d), (0, 0)), 'constant',
                            constant_values=(-1, -1))  # Pad the images so the are 32x32

            feed_dict = {img: xs, gt: ys, z: zs, mask: ms, style: ss}
            # Update the discriminator
            _, dLoss = sess.run([d_optimizer, d_loss], feed_dict=feed_dict)  

            # Update the generator, twice for good measure.
            _ = sess.run([g_optimizer], feed_dict=feed_dict)
            

            _, gLoss, advL, styleL, contL, tvL = sess.run([g_optimizer, g_loss, g_loss_adversarial, g_loss_style, g_loss_content, g_loss_tv], feed_dict=feed_dict) 
            
            print ("[Epoch: %2d / %2d] [%4d]Gen Loss: %.4f Disc Loss: %.4f, style: %.4f, content: %.4f, adv: %.4f, tv: %.4f") \
                  % (epoch, max_epoch, batchNum, gLoss, dLoss, styleL, contL, advL, tvL)
            with open(model_directory + '/training_log.txt', 'a') as text_file:
                text_file.write("[Epoch: %2d / %2d] [%4d]Gen Loss: %.4f Disc Loss: %.4f, style: %.4f, content: %.4f, adv: %.4f, tv: %.4f \n"% (epoch, max_epoch, batchNum, gLoss, dLoss, styleL, contL, advL, tvL))
            batchNum += 1
            if training_util.global_step(sess, global_step) % 100 == 0:

                img_sample, gt_sample, mask_sample = Net.matTonpy_35()

                z2 = np.random.normal(0, 1.0, size=[batch_size, z_size]).astype(np.float32)                

                # img_sample = img_sample[:, :, :, 1]
                img_sample = (np.reshape(img_sample, [sample_batch, img_x, img_y, channel]) - 0.5) * 2.0
                # Pad the images so the are 32x32
                img_sample = np.lib.pad(img_sample, ((0, 0), (padding_l, padding_r), (padding_t, padding_d), (0, 0)),
                                        'constant', constant_values=(-1, -1))

                gt_sample = (np.reshape(gt_sample, [sample_batch, img_x, img_y, 1]) - 0.5) * 2.0
                # Pad the images so the are 32x32
                gt_sample = np.lib.pad(gt_sample, ((0, 0), (padding_l, padding_r), (padding_t, padding_d), (0, 0)),
                                       'constant', constant_values=(-1, -1))

                mask_sample = (np.reshape(mask_sample, [sample_batch, img_x, img_y, 1]) - 0.5) * 2.0
                # Pad the images so the are 32x32
                mask_sample = np.lib.pad(mask_sample, ((0, 0), (padding_l, padding_r), (padding_t, padding_d), (0, 0)),
                                        'constant', constant_values=(-1, -1))

                z3 = np.random.normal(0, 1.0, size=[batch_size, z_size]).astype(np.float32)
                
                sa = 0
                sb = 1
                sc = 2
                sd = 3

                syn_sample_a, dLreal_val_a, dLfake_val_a = sess.run([syn, Dx, Dg],
                                                                    feed_dict={img: [img_sample[sa]], gt: [gt_sample[sa]], z: z2, mask:[mask_sample[sa]]})
                syn_sample_b, dLreal_val_b, dLfake_val_b = sess.run([syn, Dx, Dg],
                                                                    feed_dict={img: [img_sample[sb]], gt: [gt_sample[sb]], z: z3, mask:[mask_sample[sb]]})
                syn_sample_c, dLreal_val_c, dLfake_val_c = sess.run([syn, Dx, Dg],
                                                                    feed_dict={img: [img_sample[sc]], gt: [gt_sample[sc]], z: z2, mask:[mask_sample[sc]]})
                syn_sample_d, dLreal_val_d, dLfake_val_d = sess.run([syn, Dx, Dg],
                                                                    feed_dict={img: [img_sample[sd]], gt: [gt_sample[sd]], z: zs, mask:[mask_sample[sd]]})

                syn_sample = np.concatenate((syn_sample_a, syn_sample_b, syn_sample_c,syn_sample_d),axis=0)

                syn_sample_am = syn_sample_a * ((mask_sample[sa] + 1) / 2)
                syn_sample_bm = syn_sample_b * ((mask_sample[sb] + 1) / 2)
                syn_sample_cm = syn_sample_c * ((mask_sample[sc] + 1) / 2)
                syn_sample_dm = syn_sample_d * ((mask_sample[sd] + 1) / 2)
                syn_sample_m = np.concatenate((syn_sample_am, syn_sample_bm, syn_sample_cm, syn_sample_dm), axis=0)
                
                dLreal_val = (dLreal_val_a + dLreal_val_b + dLreal_val_c + dLreal_val_d) / 4
                dLfake_val = (dLfake_val_a + dLfake_val_b + dLfake_val_c + dLfake_val_d) / 4

                # Save sample generator images for viewing training progress.
                save_images(np.reshape(syn_sample, [sample_batch, img_x, img_y, channel]),
                            [int(np.sqrt(sample_batch)), int(np.sqrt(sample_batch))],
                            sample_directory + '/fig' + str(training_util.global_step(sess, global_step)) + '.png')
                            
                save_images(np.reshape(syn_sample_m, [sample_batch, img_x, img_y, channel]),
                            [int(np.sqrt(sample_batch)), int(np.sqrt(sample_batch))],
                            sample_directory2 + '/fig' + str(training_util.global_step(sess, global_step)) + '.png')

                print ("[Sample (global_step = %d)] real: %.4f fake: %.4f" \
                      % (training_util.global_step(sess, global_step), np.mean(dLreal_val), np.mean(dLfake_val)))
                with open(model_directory + '/training_log.txt', 'a') as text_file:
                    text_file.write("[Sample (global_step = %d)] real: %.4f fake: %.4f \n"
                                    % (training_util.global_step(sess, global_step), np.mean(dLreal_val),
                                       np.mean(dLfake_val)))

            if training_util.global_step(sess, global_step) % 1000 == 0:
                saver.save(sess,
                           model_directory + '/model-' + str(training_util.global_step(sess, global_step)) + '.cptk')
                print ("Saved Model %d, time: %.4f" % (training_util.global_step(sess, global_step), time.time()-stime))

             

saver.save(sess, model_directory + '/model-' + str(training_util.global_step(sess, global_step)) + '.cptk')
print ("Saved Model %d, time: %.4f" % (training_util.global_step(sess, global_step), time.time()-stime))

sess.close()

Instructions for updating:
Colocations handled automatically by placer.


ValueError: ignored

# dataBlocks.py

In [22]:
import numpy as np
from multiprocessing import Process, Queue


class DataIterator(object):
    def __init__(self, *data, **params):
        '''
        PARAMS:
            fullbatch (bool): decides if the number of examples return after every
                              iteration should be always a full batch.
        '''
        self.data = data
        self.batchsize = params['batchsize']
        if 'fullbatch' in params:
            self.fullbatch = params['fullbatch']
        else:
            self.fullbatch = False

    def __iter__(self):
        self.first = 0
        return self

    def __len__(self):
        return len(self.data[0])

    def __getitem__(self, key):
        outs = []
        for val in self.data:
            outs.append(val[key])
        return self.__class__(*outs, batchsize=self.batchsize, fullbatch=self.fullbatch)


class SequentialIterator(DataIterator):
    '''
    batchsize = 3
    [0, 1, 2], [3, 4, 5], [6, 7, 8]
    '''
    def next(self):
        if self.fullbatch and self.first+self.batchsize > len(self):
            raise StopIteration()
        elif self.first >= len(self):
            raise StopIteration()

        outs = []
        for val in self.data:
            outs.append(val[self.first:self.first+self.batchsize])
        self.first += self.batchsize
        return outs


class StepIterator(DataIterator):
    '''
    batchsize = 3
    step = 1
    [0, 1, 2], [1, 2, 3], [2, 3, 4]
    '''
    def __init__(self, *data, **params):
        super(self, StepIterator).__init__(self, *data, **params)
        self.step = params['step']

    def next(self):
        if self.fullbatch and self.first+self.batchsize > len(self):
            raise StopIteration()
        elif self.first >= len(self):
            raise StopIteration()

        outs = []
        for val in self.data:
            outs.append(val[self.first:self.first+self.batchsize])
        self.first += self.step
        return outs


def np_load_func(path):
    with open(path) as fin:
        arr = np.load(fin)
    return arr


class DataBlocks(object):

    def __init__(self, data_paths, batchsize=32, load_func=np_load_func, allow_preload=False):
        """
        DESCRIPTION:
            This is class for processing blocks of data, whereby dataset is loaded
            and unloaded into memory one block at a time.
        PARAM:
            data_paths (list or list of list): contains list of paths for data loading,
                            example:
                                [f1a.npy, f1b.npy, f1c.npy]  or
                                [(f1a.npy, f1b.npy, f1c.npy), (f2a.npy, f2b.npy, f2c.npy)]
            load_func (function): function for loading the data_paths, default to
                            numpy file loader
            allow_preload (bool): by allowing preload, it will preload the next data block
                            while training at the same time on the current datablock,
                            this will reduce time but will also cost more memory.
        """

        assert isinstance(data_paths, (list)), "data_paths is not a list"
        self.data_paths = data_paths
        self.batchsize = batchsize
        self.load_func = load_func
        self.allow_preload = allow_preload
        self.q = Queue()


    def __iter__(self):
        self.files = iter(self.data_paths)
        if self.allow_preload:
            self.lastblock = False
            bufile = next(self.files)
            self.load_file(bufile, self.q)
        return self


    def next(self):
        if self.allow_preload:
            if self.lastblock:
                raise StopIteration

            try:
                arr = self.q.get(block=True, timeout=None)
                self.iterator = SequentialIterator(*arr, batchsize=self.batchsize)
                bufile = next(self.files)
                p = Process(target=self.load_file, args=(bufile, self.q))
                p.start()
            except:
                self.lastblock = True
        else:
            fpath = next(self.files)
            arr = self.load_file(fpath)
            self.iterator = SequentialIterator(*arr, batchsize=self.batchsize)

        return self.iterator


    def load_file(self, paths, queue=None):
        '''
        paths (list or str): []
        '''
        data = []
        if isinstance(paths, (list, tuple)):
            for path in paths:
                data.append(self.load_func(path))
        else:
            data.append(self.load_func(paths))
        if queue:
            queue.put(data)
        return data


    @property
    def nblocks(self):
        return len(self.data_paths)


class SimpleBlocks(object):

    def __init__(self, data_paths, batchsize=32, load_func=np_load_func, allow_preload=False):
        """
        DESCRIPTION:
            This is class for processing blocks of data, whereby dataset is loaded
            and unloaded into memory one block at a time.
        PARAM:
            data_paths (list or list of list): contains list of paths for data loading,
                            example:
                                [f1a.npy, f1b.npy, f1c.npy]  or
                                [(f1a.npy, f1b.npy, f1c.npy), (f2a.npy, f2b.npy, f2c.npy)]
            load_func (function): function for loading the data_paths, default to
                            numpy file loader
            allow_preload (bool): by allowing preload, it will preload the next data block
                            while training at the same time on the current datablock,
                            this will reduce time but will also cost more memory.
        """

        assert isinstance(data_paths, (list)), "data_paths is not a list"
        self.data_paths = data_paths
        self.batchsize = batchsize
        self.load_func = load_func
        self.allow_preload = allow_preload
        self.q = Queue()


    def __iter__(self):
        self.files = iter(self.data_paths)
        if self.allow_preload:
            self.lastblock = False
            bufile = next(self.files)
            self.load_file(bufile, self.q)
        return self


    def next(self):
        if self.allow_preload:
            if self.lastblock:
                raise StopIteration

            try:
                arr = self.q.get(block=True, timeout=None)
                self.iterator = SequentialIterator(*arr, batchsize=self.batchsize)
                bufile = next(self.files)
                p = Process(target=self.load_file, args=(bufile, self.q))
                p.start()
            except:
                self.lastblock = True
        else:
            fpath = next(self.files)
            arr = self.load_file(fpath)
            self.iterator = SequentialIterator(*arr, batchsize=self.batchsize)

        return self.iterator


    def load_file(self, paths, queue=None):
        '''
        paths (list or str): []
        '''
        data = []
        if isinstance(paths, (list, tuple)):
            for path in paths:
                data.append(self.load_func(path))
        else:
            data.append(self.load_func(paths))
        if queue:
            queue.put(data)
        return data


    @property
    def nblocks(self):
        return len(self.data_paths)


class DataBlocks(SimpleBlocks):

    def __init__(self, data_paths, train_valid_ratio=[5,1], batchsize=32, load_func=np_load_func, allow_preload=False):
        """
        DESCRIPTION:
            This is class for processing blocks of data, whereby dataset is loaded
            and unloaded into memory one block at a time.
        PARAM:
            data_paths (list or list of list): contains list of paths for data loading,
                            example:
                                [f1a.npy, f1b.npy, f1c.npy]  or
                                [(f1a.npy, f1b.npy, f1c.npy), (f2a.npy, f2b.npy, f2c.npy)]
            load_func (function): function for loading the data_paths, default to
                            numpy file loader
            allow_preload (bool): by allowing preload, it will preload the next data block
                            while training at the same time on the current datablock,
                            this will reduce time but will also cost more memory.
        """

        assert isinstance(data_paths, (list)), "data_paths is not a list"
        self.data_paths = data_paths
        self.train_valid_ratio = train_valid_ratio
        self.batchsize = batchsize
        self.load_func = load_func
        self.allow_preload = allow_preload
        self.q = Queue()


    def next(self):
        if self.allow_preload:
            if self.lastblock:
                raise StopIteration

            try:
                train, valid = self.q.get(block=True, timeout=None)
                self.train_iterator = SequentialIterator(*train, batchsize=self.batchsize)
                self.valid_iterator = SequentialIterator(*valid, batchsize=self.batchsize)
                bufile = next(self.files)
                p = Process(target=self.load_file, args=(bufile, self.q))
                p.start()
            except:
                self.lastblock = True
        else:
            fpath = next(self.files)
            train, valid = self.load_file(fpath)
            self.train_iterator = SequentialIterator(*train, batchsize=self.batchsize)
            self.valid_iterator = SequentialIterator(*valid, batchsize=self.batchsize)
        return self.train_iterator, self.valid_iterator


    def load_file(self, paths, queue=None):
        '''
        paths (list or str): []
        '''
        train = []
        valid = []
        if isinstance(paths, (list, tuple)):
            for path in paths:
                X = self.load_func(path)
                num_train = len(X) * self.train_valid_ratio[0] * 1.0 / sum(self.train_valid_ratio)
                num_train = int(num_train)
                train.append(X[:num_train])
                valid.append(X[num_train:])
        else:
            X = self.load_func(paths)
            # np.random.shuffle(X)
            num_train = len(X) * self.train_valid_ratio[0] * 1.0 / sum(self.train_valid_ratio)
            num_train = int(num_train)
            train.append(X[:num_train])
            valid.append(X[num_train:])
        data = [train, valid]
        if queue:
            queue.put(data)
        return data